# Práctica 2: Clasificación
## Autor: Lucía Saiz Lapique
### Fecha: 18/04/2020

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm #Para la barrita
tqdm.pandas(desc="progress-bar")
import nltk, re, pprint
from nltk import tokenize # a tweet tokenizer from nltk.

import spacy
from spacy import displacy
from spacy.lang.es.stop_words import STOP_WORDS

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor

import collections
import itertools

C:\Users\Luli\Documents\Anaconda\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# MÉTODO NLTK

1. Lea el contenido del fichero csv en un DataFrame. Se sugiere utilizar la función pandas.read_csv. Atención a la codificación de los datos entrantes.

In [2]:
datos = pd.read_csv('hotel.csv', encoding = 'UTF-8')
datos = datos.sample(frac = 1).reset_index(drop = True) ## como los datos estan ordenados por valoracion, decidimos aleatorizarlos
## para evitar overfitting

In [3]:
datos.head()

,text,label
0,El hotel es muy lindo; cada abitación con su p...,5
1,Me encantaron las habitaciones y la apariencia...,5
2,La verdad ha sido una experiencia única; habit...,3
3,Gran Hotel; impecable sus instalaciones; el pe...,5
4,Es la mejor opción para alojarte en asunción; ...,5


2. Realice el pre-procesamiento que considere necesario. Puede utilizar funciones de la librería NLTK o spaCy, a su voluntad. Recomendamos una escritura modular del código, para poder hacer pruebas posteriormente, viendo si se obtienen mejores resultados al utilizar stop-words, al realizar una extracción de formas canónicas, etc.

### Tokenizamos con NLTK

In [4]:
# introducimos el texto en una variable para tokenizarlo
texto = datos.text
print(texto)

0      El hotel es muy lindo; cada abitación con su p...
1      Me encantaron las habitaciones y la apariencia...
2      La verdad ha sido una experiencia única; habit...
3      Gran Hotel; impecable sus instalaciones; el pe...
4      Es la mejor opción para alojarte en asunción; ...
                             ...                        
195    Muy buena excelente; muy buena ubicación; cerc...
196    Muy bonita experiencia la estadía en el Hotel;...
197    Excelente atención; instalaciones; comodidad; ...
198    Estadía súper relajada y placentera. Impecable...
199    Seria bueno dejar tazas para el cafe al alcanz...
Name: text, Length: 200, dtype: object


In [5]:
max_features = 50000

In [6]:
spanish_stopwords = stopwords.words('spanish')

In [7]:
## funcion que me ha pasado miguel para tokenizar
def analizador_nltk(texto):
    vectorizador = CountVectorizer(min_df = 10, max_df = 1.0, stop_words = spanish_stopwords, max_features = max_features)
    analyze = vectorizador.build_analyzer()
    tokens = analyze(texto)
    return tokens

In [8]:
# se crea la funcion token donde se tienen en cuenta las palabras principales
datos['tokens'] = datos['text'].progress_map(analizador_nltk)
datos.reset_index(inplace=True)
datos.drop('index', inplace=True, axis=1)
datos

progress-bar: 100%|█████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 973.47it/s]


,text,label,tokens
0,El hotel es muy lindo; cada abitación con su p...,5,"[hotel, lindo, cada, abitación, proprio, estil..."
1,Me encantaron las habitaciones y la apariencia...,5,"[encantaron, habitaciones, apariencia, hotel, ..."
2,La verdad ha sido una experiencia única; habit...,3,"[verdad, sido, experiencia, única, habitacione..."
3,Gran Hotel; impecable sus instalaciones; el pe...,5,"[gran, hotel, impecable, instalaciones, person..."
4,Es la mejor opción para alojarte en asunción; ...,5,"[mejor, opción, alojarte, asunción, buenas, in..."
...,...,...,...
195,Muy buena excelente; muy buena ubicación; cerc...,5,"[buena, excelente, buena, ubicación, cerca, ce..."
196,Muy bonita experiencia la estadía en el Hotel;...,5,"[bonita, experiencia, estadía, hotel, mucha, a..."
197,Excelente atención; instalaciones; comodidad; ...,5,"[excelente, atención, instalaciones, comodidad..."
198,Estadía súper relajada y placentera. Impecable...,5,"[estadía, súper, relajada, placentera, impecab..."


3. Division del dataset en entrenamiento y test:

In [9]:
# División en conjunto de entrenamiento y test

Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(datos['tokens'], datos['label'], test_size = 0.25)

In [10]:
# Codificación de etiquetas (categorías)
# Las categorías tienen una etiqueta tipo string, pero los algoritmos de clasifiación necesitan un valor numérico

Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

4. Convierta el corpus de documentos en una matriz TF-idf. Lo más cómodo es utilizar el vectorizador TfidfVectorizer, que forma parte de sklearn. ¿Tiene influencia en el resultado final el número máximo de features a utilizar?

In [11]:
# Vectorización: consitución de matrices tf-idf a partir de los subconjuntos de entrenamiento y test

Tfidf_vect = TfidfVectorizer(analyzer = lambda x: x, min_df = 1)
Tfidf_vect.fit(datos['tokens'])

TfidfVectorizer(analyzer=<function <lambda> at 0x00000239D58043A8>,
                binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [12]:
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [13]:
Train_X_Tfidf

<150x2284 sparse matrix of type '<class 'numpy.float64'>'
	with 4389 stored elements in Compressed Sparse Row format>

Sí que influye

5. Llegados a este punto, realice modelos de entrenamiento al menos con algoritmos de clasificador bayesiano ingenuo y máquinas SVM. Obtenga resultados de precisión de la clasificación, así como las matrices de confusión para ambos modelos.

### Modelo Bayesiano

In [14]:
# Algoritmo - Naive Bayes
# Se añade al clasificador el subconjunto de entrenamiento
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf, Train_Y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [15]:
# Se generan las predicciones
predictions_NB = Naive.predict(Test_X_Tfidf)

# Y se obtiene la precisión del modelo 
print("Naive Bayes Accuracy Score -> ", accuracy_score(predictions_NB, Test_Y) * 100)
print("Naive Bayes Confusion Matrix:")
print(confusion_matrix(Test_Y, predictions_NB))

Naive Bayes Accuracy Score ->  72.0
Naive Bayes Confusion Matrix:
[[25  3]
 [11 11]]


### Modelo Support Vector Machine

In [16]:
# Algoritmo - SVM
# Se añade al clasificador el subconjunto de entrenamiento
SVM = svm.SVC(C = 1.0, kernel = 'linear', degree = 3, gamma = 'auto')
SVM.fit(Train_X_Tfidf, Train_Y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [17]:
# Se generan las predicciones
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Y se obtiene la precisión del modelo 
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, Test_Y)*100)
print("SVM Matrix:")
print(confusion_matrix(Test_Y, predictions_SVM))

SVM Accuracy Score ->  68.0
SVM Matrix:
[[23  5]
 [11 11]]


### Modelo Random Forest

In [18]:
#Lanzamos el modelo con 1000 arboles de decision
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# entrenamos el modelo con los datos de entrenamiento
rf.fit(Train_X_Tfidf, Train_Y);

In [19]:
# Se generan las predicciones
predictions_RF = rf.predict(Test_X_Tfidf)
predictions_RF2 = predictions_RF.round()
predictions_RF2

array([0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1.])

In [20]:
# Y se obtiene la precisión del modelo 
print("RF Accuracy Score -> ", accuracy_score(predictions_RF2, Test_Y)*100)
print("RF Matrix:")
print(confusion_matrix(Test_Y, predictions_RF2))

RF Accuracy Score ->  70.0
RF Matrix:
[[20  8]
 [ 7 15]]


# MÉTODO SPACY

1. Lea el contenido del fichero csv en un DataFrame. Se sugiere utilizar la función pandas.read_csv. Atención a la codificación de los datos entrantes.

In [21]:
datos2 = pd.read_csv('hotel.csv', encoding = 'UTF-8')
datos2 = datos2.sample(frac = 1).reset_index(drop = True) ## como los datos estan ordenados por valoracion, decidimos aleatorizarlos
## para evitar overfitting

In [22]:
datos.head()

,text,label,tokens
0,El hotel es muy lindo; cada abitación con su p...,5,"[hotel, lindo, cada, abitación, proprio, estil..."
1,Me encantaron las habitaciones y la apariencia...,5,"[encantaron, habitaciones, apariencia, hotel, ..."
2,La verdad ha sido una experiencia única; habit...,3,"[verdad, sido, experiencia, única, habitacione..."
3,Gran Hotel; impecable sus instalaciones; el pe...,5,"[gran, hotel, impecable, instalaciones, person..."
4,Es la mejor opción para alojarte en asunción; ...,5,"[mejor, opción, alojarte, asunción, buenas, in..."


2. Realice el pre-procesamiento que considere necesario. Puede utilizar funciones de la librería NLTK o spaCy, a su voluntad. Recomendamos una escritura modular del código, para poder hacer pruebas posteriormente, viendo si se obtienen mejores resultados al utilizar stop-words, al realizar una extracción de formas canónicas, etc.

Tokenizamos ahora con spaCy

In [23]:
# Importamos las librerías que corresponda
# Hay que añadir los modelos que se utilicen. Previamente cargados con python -m spacy download es_core_news_md

nlp_esp = spacy.load('es_core_news_md')

In [24]:
def analizador_spacy(texto):
    docum_analizado = nlp_esp(texto)
    tokens = [palabra.text for palabra in docum_analizado]
    frases_token = []
    for word in tokens:
        palabras = nlp_esp.vocab[word]
        if (palabras.is_stop == False) & (palabras.is_punct == False):
            frases_token.append(word) 
    return frases_token   

In [25]:
# se crea la funcion token donde se tienen en cuenta las palabras principales
datos2['tokens'] = datos2['text'].progress_map(analizador_spacy)
datos2.reset_index(inplace = True)
datos2.drop('index', inplace = True, axis = 1)
datos2

progress-bar: 100%|██████████████████████████████████████████████████████████████████| 200/200 [00:05<00:00, 35.01it/s]


,text,label,tokens
0,Seria bueno dejar tazas para el cafe al alcanz...,5,"[Seria, dejar, tazas, cafe, alcanze, huespedes..."
1,Colchon increíble. Habitación con todos los de...,5,"[Colchon, increíble, Habitación, detalles, esp..."
2,Precioso hotel y excelente atencion. Muy bien ...,3,"[Precioso, hotel, y, excelente, atencion, ubic..."
3,Viajo asiduamente de negocios a Asunción y sie...,5,"[Viajo, asiduamente, negocios, a, Asunción, y,..."
4,Hace unos días participamos de un desayuno de ...,3,"[participamos, desayuno, periodistas, y, quedé..."
...,...,...,...
195,Siempre es un placer hospedarse en este hotel;...,5,"[placer, hospedarse, hotel, atención, excelent..."
196,Me hospede por 2 noches en este hotel; fui de ...,3,"[hospede, 2, noches, hotel, compañera, quedamo..."
197,Excelente experiencia para los negocios o en f...,3,"[Excelente, experiencia, negocios, o, familia,..."
198,Está ubicado en una linda zona; frente a un sh...,5,"[ubicado, linda, zona, frente, a, shopping, gr..."


3. Division del dataset en entrenamiento y test:

In [26]:
# División en conjunto de entrenamiento y test

Train_X2, Test_X2, Train_Y2, Test_Y2 = model_selection.train_test_split(datos2['tokens'], datos2['label'], test_size = 0.25)

In [27]:
# Codificación de etiquetas (categorías)
# Las categorías tienen una etiqueta tipo string, pero los algoritmos de clasifiación necesitan un valor numérico

Encoder2 = LabelEncoder()
Train_Y2 = Encoder2.fit_transform(Train_Y2)
Test_Y2 = Encoder2.fit_transform(Test_Y2)

4. Convierta el corpus de documentos en una matriz TF-idf. Lo más cómodo es utilizar el vectorizador TfidfVectorizer, que forma parte de sklearn. ¿Tiene influencia en el resultado final el número máximo de features a utilizar?

In [28]:
# Vectorización: consitución de matrices tf-idf a partir de los subconjuntos de entrenamiento y test

Tfidf_vect2 = TfidfVectorizer(analyzer = lambda x: x, min_df = 1, max_features = 5000)
Tfidf_vect2.fit(datos2['tokens'])

TfidfVectorizer(analyzer=<function <lambda> at 0x00000239D5937168>,
                binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [29]:
Train_X_Tfidf2 = Tfidf_vect2.transform(Train_X2)
Test_X_Tfidf2 = Tfidf_vect2.transform(Test_X2)

5. Llegados a este punto, realice modelos de entrenamiento al menos con algoritmos de clasificador bayesiano ingenuo y máquinas SVM. Obtenga resultados de precisión de la clasificación, así como las matrices de confusión para ambos modelos.

### Modelo Bayesiano

In [30]:
# Algoritmo - Naive Bayes
# Se añade al clasificador el subconjunto de entrenamiento
Naive2 = naive_bayes.MultinomialNB()
Naive2.fit(Train_X_Tfidf2, Train_Y2)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [31]:
# Se generan las predicciones
predictions_NB2 = Naive2.predict(Test_X_Tfidf2)

# Y se obtiene la precisión del modelo 
print("Naive Bayes Accuracy Score -> ", accuracy_score(predictions_NB2, Test_Y2)*100)
print("Naive Bayes Confusion Matrix:")
print(confusion_matrix(Test_Y2, predictions_NB2))

Naive Bayes Accuracy Score ->  57.99999999999999
Naive Bayes Confusion Matrix:
[[25  0]
 [21  4]]


### Modelo Support Vector Machine

In [32]:
# Algoritmo - SVM
# Se añade al clasificador el subconjunto de entrenamiento
SVM2 = svm.SVC(C = 1.0, kernel = 'linear', degree = 3, gamma = 'auto')
SVM2.fit(Train_X_Tfidf2, Train_Y2)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [33]:
# Se generan las predicciones
predictions_SVM2 = SVM2.predict(Test_X_Tfidf2)

# Y se obtiene la precisión del modelo 
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM2, Test_Y2) * 100)
print("SVM Matrix:")
print(confusion_matrix(Test_Y2, predictions_SVM2))

SVM Accuracy Score ->  62.0
SVM Matrix:
[[20  5]
 [14 11]]


### Modelo Random Forest

In [34]:
#Lanzamos el modelo con 1000 arboles de decision
rf2 = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# entrenamos el modelo con los datos de entrenamiento
rf2.fit(Train_X_Tfidf2, Train_Y2);

In [35]:
# Se generan las predicciones
predictions_RF3 = rf2.predict(Test_X_Tfidf2)
predictions_RF4 = predictions_RF3.round()
predictions_RF4

array([0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0.])

In [36]:
# Y se obtiene la precisión del modelo 
print("RF Accuracy Score -> ", accuracy_score(predictions_RF4, Test_Y2)*100)
print("RF Matrix:")
print(confusion_matrix(Test_Y2, predictions_RF4))

RF Accuracy Score ->  60.0
RF Matrix:
[[16  9]
 [11 14]]


# COMPARATIVA FINAL

## NLTK

In [37]:
print("Naive Bayes Accuracy Score -> ", accuracy_score(predictions_NB, Test_Y)*100)
print("Naive Bayes Confusion Matrix:")
print(confusion_matrix(Test_Y, predictions_NB))

print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, Test_Y) * 100)
print("SVM Matrix:")
print(confusion_matrix(Test_Y, predictions_SVM))

print("RF Accuracy Score -> ", accuracy_score(predictions_RF2, Test_Y)*100)
print("RF Matrix:")
print(confusion_matrix(Test_Y, predictions_RF2))

Naive Bayes Accuracy Score ->  72.0
Naive Bayes Confusion Matrix:
[[25  3]
 [11 11]]
SVM Accuracy Score ->  68.0
SVM Matrix:
[[23  5]
 [11 11]]
RF Accuracy Score ->  70.0
RF Matrix:
[[20  8]
 [ 7 15]]


## SpaCy

In [38]:
print("Naive Bayes Accuracy Score -> ", accuracy_score(predictions_NB2, Test_Y2)*100)
print("Naive Bayes Confusion Matrix:")
print(confusion_matrix(Test_Y2, predictions_NB2))

print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM2, Test_Y2) * 100)
print("SVM Matrix:")
print(confusion_matrix(Test_Y2, predictions_SVM2))

print("RF Accuracy Score -> ", accuracy_score(predictions_RF4, Test_Y2)*100)
print("RF Matrix:")
print(confusion_matrix(Test_Y2, predictions_RF4))

Naive Bayes Accuracy Score ->  57.99999999999999
Naive Bayes Confusion Matrix:
[[25  0]
 [21  4]]
SVM Accuracy Score ->  62.0
SVM Matrix:
[[20  5]
 [14 11]]
RF Accuracy Score ->  60.0
RF Matrix:
[[16  9]
 [11 14]]


6. Comente los resultados obtenidos. ¿Qué factores influyen? ¿Los resultados obtenidos son los esperados inicialmente? ¿A qué se deben estos resultados? Piense en la calidad del conjunto de datos con los que está trabajando

__¿Qué factores influyen?__

* El tamaño de la muestra de test (si se aumenta esta muestra, mejora el accuracy)
* Poner semilla o no
* La cantidad de datos que hay, ya que, al ser la base de datos tan pequeña, los resultados de la precisión de predicción de cada modelo van cambiando.
* Que los datos estén ordenados por calificación o no (al estar ordenados, se genera overfitting; por ello, al principio se decide aleatorizar el orden en el que estén)
* Con cada iteración, cambian los resultados

__¿Los resultados obtenidos son los esperados inicialmente?__

Los resultados obtenidos no son los esperados, en primer lugar porque se esperaba que los reusltados de spaCy fuesen mejores que con el método de NLTK, y no ha sido así. El accuracy en todos los métodos aplicados es mejor para los tres modelos a través de NLTK que de spaCy. En los tres casos de spaCy es bastante más bajo, en alguna de las iteraciones generadas incluso no es capaz de llegar a un 60% en ninguno de los modelos.

En cuanto a la selección del mejor método, se han utilizado el modelo Bayesiano, un Support Vector Machine y un Random Forest. Tras ejecutar los tres modelos para cada método, hemos realizado una comparativa de los tres modelos con cada método, comparando el accuracy y la matriz de confusión de cada uno. Analizando los resultados predictivos de los tres, para ambos métodos, el modelo que devuelve el valor más alto en accuracy y mejor matriz es el modelo Bayesiano para el método de NLTK, y el Support Vector Machine para spaCy. 

Sin embargo, estos datos no resultan ser muy de fiar, ya que con cada nuevo intento de ejecución del notebook, salen resultados distintos para cada modelo y cada método. 

__¿A qué se deben estos resultados?__

Los resultados obtenidos y los problemas que han surgido (que la precisión vaya cambiando con cada iteración, que sea muy baja, etc.), se debe probablemente a que la base de datos con la que estamos trabajando es muy pequeña y, por tanto, no tiene registros suficientes para generar un modelo capacitado para predecir lo que necesitamos. Para mejorarlo, habría que ampliar la base de datos y que la muestra de train y test sean mayores. 

Otra cosa que se tuvo que alterar fue el orden de los registros, pues nos dimos cuenta de que estaban ordenados por la nota del label, con lo cual se generaba cierto overfitting. Aunque desordenando los datos el resultado de la precisión de predicción es menor, ahora es más fiable. 